### Restaurantes

Dada una url de una página de restaurantguru, la inspecciona y extrae las caracteriísticas más relevantes del establecimiento.

· Nombre

· Coordenadas (latitud y longitud) y localidad en la que está.

· Rango de precio (por persona)

· Contacto (número de teléfono)

· Horario (ordenados por día de la semana de lunes a domingo, incluido si está cerrado)

· Descripción (depende mucho del restaurante, hay algunas con mucho detalle y otras muy simples)

· Tipos de comida (mediterránea, española, ...)

· Servicios (terraza, wifi, para llevar, ...)

· Valoración: valoración sobre 100 que proviene de las opiniones

· Número de votos: número de opiniones registradas

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import re
import json
import pandas as pd
import time
import random
from bs4 import BeautifulSoup,SoupStrainer  
import requests

# 1.Selenium

Funciones auxiliares para obtener los enlaces

In [ ]:
def process_bs(html):
    """
    Función para extraer info relevante (enlaces,la valoración media y el número de opiniones) dado un html

    Parámetros:
    - html

    Salida:
    - Listas con los enlaces,la valoración media y el número de opiniones.
    """
    #parsear con bs
    soup = BeautifulSoup(html, "lxml", 
                     parse_only = SoupStrainer( 
                       'div', class_ = 'restaurant_container'))
    #encontrar todos los wrapers que contiennen info
    wrappers=soup.find_all('div',class_='wrapper_info')
    urls = []
    estrellas = []
    n_comentarios = []
    ######################## LOOP WRAPERS ####################
    for wrapper in wrappers:
        href=wrapper.find('a', class_="notranslate title_url").get('href')#urls
        urls.append(href)
        try:
            valoracion=wrapper.find('div', class_="rating-stars__fill").get("style") # Valoración
            n_opiniones = wrapper.find('span', class_="rating-stars__text").text # Número de comentarios
            estrellas.append(valoracion)
            n_comentarios.append(n_opiniones)
        #si no se encuentran los elementos los rellenamos con 0
        except:
            estrellas.append('width: 0%;')
            n_comentarios.append('0 votos')

    return urls,estrellas,n_comentarios
def process_selenium(driver):
    """
    Función para extraer info relevante (enlaces,la valoración media y el número de opiniones) dado un driver

    Parámetros:
    - driver

    Salida:
    - Listas con los enlaces,la valoración media y el número de opiniones.
    """
    urls = []
    estrellas = []
    n_comentarios = []
    #todos los elementos wrpapers que contienen la info
    wrappers = driver.find_elements(By.XPATH, '//div[@class="wrapper_info"]')
    ######################## LOOP WRAPERS ####################
    for wrapper in wrappers:
        href = wrapper.find_element(By.XPATH, './/a[@class="notranslate title_url"]').get_attribute('href')#urls
        urls.append(href)
        try:
            valoracion=wrapper.find_element(By.CLASS_NAME, "rating-stars__fill").get_attribute("style") # Valoración
            estrellas.append(valoracion)
            n_opiniones = wrapper.find_element(By.CLASS_NAME, "rating-stars__text").text # Número de comentarios
            n_comentarios.append(n_opiniones)
        #si no se encuentran los elementos los rellenamos con 0
        except:
            estrellas.append('width: 0%;')
            n_comentarios.append('0 votos')
    return urls,estrellas,n_comentarios

Función 1: extraer, para una url (como driver ya inicializado) de restaurantguru general (en nuestro caso será referente a localidades de Asturias), las urls de los distintos alojamientos que se encuentran en esa zona. Además, se obtiene también la valoración media de esos restaurantes en función de los comentarios y el número de comentario que se utilizan para llegar a esa valoración.

In [49]:
def obtener_enlaces_restaurantes(driver,data_extractor='sele', max_urls = 50):
    """
    Función para obtener un número máximo de enlaces de restaurantes de una determinada zona desde una página cargada de restaurantguru.

    Parámetros:
    - driver: WebDriver de Selenium ya inicializado y posicionado en la URL deseada.
    - data_extractor: indica el extractor de datos a usar, sele/otro(bs)
    - max_urls: Número máximo de URLs a extraer.

    Salida:
    - Un dataframe con los enlaces,la valoración media y el número de opiniones.
    """

    # Almacenar urls, valoración media y número de comentarios en listas
    urls = []
    estrellas = []
    n_comentarios = []
    wait = WebDriverWait(driver, 10) # Espera
    last_height = driver.execute_script("return document.body.scrollHeight") # Altura total del contenido de la página.
    ############################# LOOP SCROLL ###############################
    number_urls=0
    while number_urls < max_urls:
        # Desplazarse hacia abajo para cargar más contenido
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight") # Nueva altura de la página
        
        # Si no hay más contenido para cargar con desplazamiento acaba el bucle
        if new_height == last_height: 
            break
        
        last_height = new_height # Actualiza la altura de la página
        number_urls+=20#cada scroll añade 20 enlaces mas
    ############### DATA EXTRACTION ##################
    if data_extractor=='sele':
        urls,estrellas,n_comentarios = process_selenium(driver)
    else:
        html = driver.page_source
        urls,estrellas,n_comentarios = process_bs(html)
    # Recortar las lista al máximo de URLs buscadas
    urls = urls[:max_urls]
    estrellas = estrellas[:max_urls]
    n_comentarios = n_comentarios[:max_urls]

    # Transforma las estrellas y el número de comentarios para que puedan ser tratados numéricamente
    estrellas = pd.to_numeric([int(cadena.split(':')[1].strip().replace('%', '').replace(';', '')) for cadena in estrellas])
    n_comentarios = pd.to_numeric([int(cadena.split(' ')[0]) for cadena in n_comentarios])

    # Construir un dataframe con los datos
    resultado = pd.DataFrame({
    'urls': urls,
    'valoracion': estrellas,
    'numero_votos': n_comentarios
    })
    return resultado

Función 2: extraer, para un determinado restaurante (como driver ya inicializado), la características y la información más relevante (se puede modificar, es una primera idea).

In [3]:
def informacion_restaurante(driver):
    """
    Función para obtener la información relevante de un determinado restaurante a partir de una página cargada de restaurantguru

    Parámetros:
    - driver: WebDriver de Selenium ya inicializado y posicionado en la URL deseada.

    Salida:
    - nombre: nombre del restaurante correspondiente.
    - lat, long: coordenadas del restaurante.
    - direccion: dirección completa del restaurante.
    - localidad: localidad (ciudad, pueblo, ...) en la que se encuentra el restaurante.
    - rango_precio: rango de precio medio por persona o precio máximo por persona del restaurante (algunos son desconocidos).
    - telefono: contacto del restaurante (con prefijo).
    - horas: horario del restaurante (en formato lista, cada día un elemento, ordenados de lunes a domingo)
    - tipo: tipos de cocina del restaurante (lista). 
    - descripcion: pequeña descripción del restaurante (su extensión y precisión depende del alojamiento, algunos no disponen de ella).
    - caracteristicas:  servicios o características que ofrece el restaurante (lista).
    
    """

    

    # Obtener el NOMBRE del restaurante
    nombre= driver.find_element(By.TAG_NAME, 'h1').text

    # Obtener las COORDENADAS del restaurante
    try:
        script_tag = driver.find_element(By.XPATH, "//script[@type='application/ld+json']").get_attribute('innerHTML')
        coord = json.loads(script_tag)
        # Accede a la latitud y longitud
        lat = coord['geo']['latitude']
        long = coord['geo']['longitude']
    except: # Si no las conocemos, las suponemos desconocidas
        lat = "Desconocida"
        long = "Desconocida"


    ## Obtener las DIRECCiÓN del restaurante
    try:
        direccion = driver.find_element(By.ID, "info_location").find_elements(By.TAG_NAME, "div")[1].text.strip()
    except: # Si no la conocemos, la suponemos desconocida
        direccion = "Desconocida"

    # Obtener la LOCALIDAD del restaurante
    try:
        elementos = driver.find_elements(By.CSS_SELECTOR, 'li[itemprop="itemListElement"] span[itemprop="name"]')
        # Extraer la localidad y eliminar "Principado de Asturias" si está presente
        localidad = re.sub(r',\s*Principado de Asturias', '', elementos[-1].text)
    except: # Si no la conocemos, la suponemos desconocida
        localidad = "Desconocida"

    # Obtener el RANGO DE PRECIOS (o PRECIO de referencia) del restaurante. Cadena de texto
    try:
        rango_precio = driver.find_element(By.XPATH, "//span[@class='nowrap']//span").text # Obtenermos el rango de precios
    except: 
        try:
            # Si el primer rango de precios no se encuentra, intentar con el segundo
            rango_precio = driver.find_element(By.XPATH, '//span[contains(@class, "text_overflow")]//span[contains(@class, "nowrap")]').text
        except:
            # Si ambos intentos fallan, asignar "Desconocido"
            rango_precio = "Desconocido"
    
    # Obtener el contacto (TELÉFONO) del restaurante
    try:
        telefono = driver.find_element(By.CSS_SELECTOR, 'a[href^="tel:"]').get_attribute('href').replace("tel:", "")
    except:
        telefono = ""
    
    
    # Obtener el HORARIO del restaurante
    try:    
        horario_dias = driver.find_element(By.CLASS_NAME, "schedule-table").find_elements(By.TAG_NAME, "tr")
        # Lista para almacenar los horarios por días
        horas = []
        for row in horario_dias:
            # Encuentra las columnas de cada día
            columns = row.find_elements(By.TAG_NAME, "td")

            # Extrae el horario (puede contener saltos de línea)
            elemento = columns[1].get_attribute("innerHTML").replace("<br>", "\n")
            horas.append(elemento)
    except:
        horas = []

    # Obtener los TIPOS de comida
    try:
        cocina = driver.find_element(By.CLASS_NAME, "cuisine_wrapper") # Encuentra el "apartado" con los tipos de cocina
        tipos_cocina = cocina.find_elements(By.TAG_NAME, "span") # Encuentra todos los tipos de cocina
        tipos = [item.text for item in tipos_cocina] # Extrae cada tipo (lista en la que cada elemento es un tipo)
    except: # Si no aparecen en la página, se considera vacía
        tipos = []

    # Obtener CARACTERÍSTICAS del restaurante (se puede modificar para que de solo las características, según nos interese)
    try:
        caracteristicas  = driver.find_element(By.CLASS_NAME, "features_block").find_elements(By.CLASS_NAME, "feature_item")
        df_caracteristicas = [{"Característica": item.get_attribute("class").split()[1], "Descripción": item.text} for item in caracteristicas]# Crea una lista con el nombre de la característica y su texto
        df_caracteristicas = pd.DataFrame(df_caracteristicas)# Crea un DataFrame con los datos obtenidos
    except: # Si no aparecen en la página, se considera vacía
        df_caracteristicas = pd.DataFrame(columns=['Característica', 'Descripción'])

    # Obtener la DESCRIPCIÓN del restaurante (hay algunas que son muy cortas o no existen)
    try:
        descripcion= driver.find_element(By.CLASS_NAME, "description").get_attribute("innerText")
    except: # Si no aparece en la página, se considera vacía
        descripcion = ''

    return nombre, lat, long, direccion, localidad, rango_precio, telefono, horas, tipos, df_caracteristicas["Descripción"].tolist(), descripcion

Partiendo de los nombre de diferentes localidades asturianas (selección propia, se pueden añadir o quitar), se construyen las URLs correspondientes a la página de restaurantguru que contiene la lista de los restaurantes de su zona de influencia. Las URLs tiene todas la misma estructura, variando unicamente la localidad seleccionada.

In [33]:
# Lista de ciudades
ciudades = ["Cangas del Narcea", "Pola de Lena", "Cangas de Onis", "Aviles", "Grado Asturias", "Pola de Siero", "Castropol", "Gijon", 
            "Pola de laviana", "Oviedo","Llanes", "Mieres Asturias", "Langreo", "Tineo", "Luarca", "Pravia", "Villaviciosa", "Pilona",
            "Ribadesella", "Navia", "Vegadeo", "Lastres", "Cabrales", "Grandas de Salime"]
ciudades = ["Colunga", "Norena", "Candas", "Luanco", "Tapia de Casariego", "Cudillero", "Soto del Barco", "Penamellera Baja", "Ribera de Arriba", "Covadonga","Caravia",
            "Teverga"]
# Buscar las urls para todas las ciudades consideradas
base_url = 'https://es.restaurantguru.com/'
urls_ciudad = []
for ciudad in ciudades:
    ciudad = ciudad.replace(" ", "-")
    url = f"{base_url}{ciudad}#restaurant-list" 
    urls_ciudad.append(url)

Una vez se dispone de todas las URLs zonales, se procede a aplicar la función obtener_enlaces_restaurantes a cada uno de ellos, obteniendo los enlaces correspondientes a los restaurantes de esa zona. Justo a cada una de los enlaces, se almacena también la valoración media del establecimiento y el número de opiniones que han llevado a ella. (En una primera ejecución, se obtienen únicamente 2 restaurantes de cada una de las zonas y tarda algo más de 3 minutos)

In [48]:
def data_assemble(url,df,driver):
    """
    Función que agrupa varios procesos para obtneer el df con todos los enlaces

    Parámetros:
    - url:  URL deseada
    - df: df al que se quiere mergear
    - driver: WebDriver de Selenium ya inicializado y posicionado en la URL deseada.

    Salida:
    - df: df total con toda la info relevante
    
    """    
    if "Oviedo" in url:
        max_urls=600
        data_extractor='bs'
    elif "Gijon" in url:
        max_urls=700
        data_extractor='bs'
    elif "Aviles" in url:
        max_urls=225
        data_extractor='bs'
    else:
        max_urls=80
        data_extractor='sele'
    
    start=time.time()
    # Extraemos los resultados
    enlaces_restaurantes = obtener_enlaces_restaurantes(driver,data_extractor, max_urls)
    print(f"{len(enlaces_restaurantes)} enlaces capturados en {url.split('#')[0].split('/')[-1]}, t={round(time.time()-start,4)}s")
    #mergeamos y filtramos
    df = pd.concat([df, enlaces_restaurantes])
    df=df[(df['valoracion']>50) & (df['numero_votos']>10)].reset_index(drop=True)
    return df

In [ ]:
# Dataframe para guardar las urls, la valoración media y el número de votos 
df = pd.DataFrame()

# Inicializamos el driver
options = Options()
options.add_argument('--headless=new')  # Ejecutar el navegador en modo headless
driver = webdriver.Chrome(options=options)

url=urls_ciudad[0]
# Obtenemos los resultados para la primera url
driver.get(url)
time.sleep(3)

# Aceptamos las cookies
try:
    cookies= driver.find_element(By.XPATH,'/html/body/div[9]/div[2]/div[2]/div[2]/div[2]/button[1]')
except:
    cookies= driver.find_element(By.XPATH,'/html/body/div[8]/div[2]/div[2]/div[2]/div[2]/button[1]')
time.sleep(1)
cookies.click()
time.sleep(1)

df=data_assemble(url,df,driver)
html = driver.page_source
# Iteramos para obtener los resultados en el resto de ciudades
for id in range(1,len(urls_ciudad)):# len(urls_ciudad)
    url=urls_ciudad[id]
    driver.execute_script(f'window.open("{url}", "_blank");') #Abrimos la nueva url
    time.sleep(1)
    driver.close() # Cerramos la url antigua
    time.sleep(0.5)
    driver.switch_to.window(driver.window_handles[0]) # Nos movemos a la nueva url
    time.sleep(3)
    df=data_assemble(url,df,driver)
driver.quit()


break by height
60 enlaces capturados en Cangas-del-Narcea, t=4.6627s
break by height
39 enlaces capturados en Pola-de-Lena, t=2.9701s
80 enlaces capturados en Cangas-de-Onis, t=6.5599s
225 enlaces capturados en Aviles, t=21.1474s
break by height
47 enlaces capturados en Grado-Asturias, t=4.126s
80 enlaces capturados en Pola-de-Siero, t=5.9435s
break by height
21 enlaces capturados en Castropol, t=2.5305s
700 enlaces capturados en Gijon, t=38.3158s
break by height
48 enlaces capturados en Pola-de-laviana, t=4.3191s
80 enlaces capturados en Llanes, t=6.9149s
80 enlaces capturados en Mieres-Asturias, t=6.6281s
80 enlaces capturados en Langreo, t=6.628s
break by height
32 enlaces capturados en Tineo, t=2.7244s
break by height
61 enlaces capturados en Luarca, t=5.764s
break by height
42 enlaces capturados en Pravia, t=4.0633s
80 enlaces capturados en Villaviciosa, t=5.9746s
break by height
19 enlaces capturados en Pilona, t=1.4178s
80 enlaces capturados en Ribadesella, t=6.277s
break by he

In [55]:
df

,urls,valoracion,numero_votos
0,https://es.restaurantguru.com/vinos-a-granel-O...,76,27
1,https://es.restaurantguru.com/Casa-Fermin-Oviedo,100,2360
2,https://es.restaurantguru.com/Cocina-Cabal-Oviedo,94,888
3,https://es.restaurantguru.com/NM-Oviedo,94,64
4,https://es.restaurantguru.com/CaSuso-Oviedo,94,2157
...,...,...,...
595,https://es.restaurantguru.com/Pescados-Joaquin...,74,40
596,https://es.restaurantguru.com/Varyarte-Oviedo,74,1110
597,https://es.restaurantguru.com/Llamaquique-Oviedo,64,16
598,https://es.restaurantguru.com/Manolo-Sidreria-...,72,134


In [ ]:
# Guardar las urls de los restaurantes (hay 2 * 24 = 48)
df.to_csv('../../Data/Data_used/restaurant_urls.csv', index=False)

Se extraen las características de cada uno de los restaurantes encontrados utilizando la función informacion_restaurante. Los datos se guardan en un dataframe junto con la valoración y el número de comentario ya encontrados previamente. (Para 48 alojamientos tarda aproximadamente 7 minutos)

In [ ]:
# Cargar los datos (urls, valoraciones y número de comentarios) de los diferenres alojamientos.
df = pd.read_csv('../../Data/Data_used/restaurant_urls.csv') 

# 2.Request

Función 1: extraer, para un determinado restaurante (como driver ya inicializado), la características y la información más relevante (se puede modificar, es una primera idea).

In [48]:
def from_responseContent_to_data(response,print_=False):
    """
    Función: 
        para obtener la información relevante de un determinado restaurante a partir de la respuesta (requests) de una página cargada de restaurantguru.
        Si no ebncuentra las coordenadas devuelve un KO en titulo para poder procesarlo luego

    Parámetros:
    - response: response.content de un request.get
    - print_: si quieres ver los datos obtenidos

    Salida:
    - nombre: nombre del restaurante correspondiente.
    - lat, long: coordenadas del restaurante.
    - loc: dirección completa del restaurante.
    - localidad: localidad (ciudad, pueblo, ...) en la que se encuentra el restaurante.
    - precios: rango de precio medio por persona o precio máximo por persona del restaurante (algunos son desconocidos).
    - telefono: contacto del restaurante (con prefijo).
    - horarios: horario del restaurante (en formato lista, cada día un elemento, ordenados de lunes a domingo)
    - tipo: tipos de cocina del restaurante (lista). 
    - descripcion: pequeña descripción del restaurante (su extensión y precisión depende del alojamiento, algunos no disponen de ella).
    - caracteristicas:  servicios o características que ofrece el restaurante (lista).
    
    """
    soup = BeautifulSoup(response, 'html.parser')
    ############## nombre ###############
    title_elelment= soup.find('div', {'class': 'title_container'})
    titulo= title_elelment.find('h1', {'class': 'notranslate'}).text.strip()
    ######################## lat y lon ###########################
    try:
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        coord = json.loads(script_tag.string)
        # Accede a la latitud y longitud
        lat = coord['geo']['latitude']
        long = coord['geo']['longitude']
    except:
        return 'KO', 'lat', 'long', 'loc', 'localidad', 'precios', 'telefono', 'horario', 'tipo_cocina', 'valora','votos','caracteristicas', 'descripcion'
    ############# location #########################
    loc_element=soup.find('div',{'id':'info_location'})
    loc=loc_element.find_all('div')[1].text.replace('\n','').strip()
    ############ localidad #################
    localidad=loc.split(',')[-3]
    ############ telefono ~######################
    telefono = soup.find('a', href=re.compile(r'^tel:')).get('href').replace('tel:','')
    ############# precios #####################
    try:
        precios=soup.find('span',{'class':'nowrap'}).text
    except:
        precios='Desconocido'
    ############ horario ###################
    try:
        horario=soup.find('table',{'class':'schedule-table'}).text.split('\n')
        horario=[item for item in horario if item!='']
    except:
        horario='Desconocido'
    ########### Tipos de cocina	 ##################
    try:
        tipo_cocina_element=soup.find('div',{'class':'cuisine_wrapper'})
        tipo_cocina_span=tipo_cocina_element.find_all('span')
        tipo_cocina=[tipo.text.replace('\n','').strip() for tipo in tipo_cocina_span]
    except:
        tipo_cocina='Desconocido'
    ############ Valoración ###############
    valora=soup.find('div',{'class':'rating-stars__fill'}).get("style").replace('width:','').replace('%','')
    ######### Número de votos #################
    votos=soup.find('span',{'class':'rating-stars__text'}).text.replace('votos','').strip()
    ########## Características ##############
    features_block=soup.find('div',{'class':'features_block'})
    spans=features_block.find_all('span')
    caracteristicas=[cara.text for cara in spans]
    ############## description #################
    #wrapper_description
    descripcion=soup.find('div',class_='description').text.replace('\n','')
    if print_==True:
        for item in [titulo, lat, long, loc, localidad, precios, telefono, horario, tipo_cocina, valora,votos,caracteristicas, descripcion]:
            print(item)

    return titulo, lat, long, loc, localidad, precios, telefono, horario, tipo_cocina, valora,votos,caracteristicas, descripcion
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
response = requests.get('https://es.restaurantguru.com/Langrehotel-Bar-Langreo', headers=headers)
titulo, lat, long, loc, localidad, precios, telefono, horario, tipo_cocina, valora,votos,caracteristicas, descripcion=from_responseContent_to_data(response.content,print_=True)

AZZ Asturias LangreHotel & SPA
43.30576580
-5.69043890
C/ Manuel Suárez García, 6, Langreo, Principado de Asturias, España
 Langreo
Desconocido
+34985675675
Desconocido
Desconocido
74
1856
['Asientos al aire libre']
AZZ Asturias LangreHotel & SPA está muy cerca de Museo de la Siderurgia de Asturias. Según las opiniones de los críticos, los camareros te sirven aquí un recomendable laing y un buen cerdo. La gente normalmente visita AZZ Asturias LangreHotel & SPA para pedir su sensacional café. El vino de este lugar no es muy del agrado de sus visitantes. Es fácil encontrar este lugar gracias a su gran ubicación. Comprueba por ti mismo lo admirable que es su personal. Te va a gustar su sofisticado servicio. Una serie de usuarios han reparado en el hecho de que la decoración es hogareña. Los usuarios de Google han premiado a AZZ Asturias LangreHotel & SPA con un 4,2.


Establecer opciones del navegador

In [6]:
option = webdriver.ChromeOptions()
#option.add_argument('--headless=new')#para que no se abra el navegador
#option.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36')
option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36")
#option.add_argument('--proxy-server=http://123.123.123.123:8080')

option.add_argument("window-size=1400,600")
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-shm-usage')    

option.add_argument("--disable-search-engine-choice-screen")#para que no se abra ventana de escoger navegador
option.add_argument("--disable-web-security") 
option.add_argument("--disable-gpu")
option.add_argument('--log-level=1')

#s = Service(ChromeDriverManager().install())

Funcion 2: para resolver el captcha de forma manual

In [24]:
def manual_captcha_resolve(url):
    """
    Función: 
        dada una url cualquiera crea una bventana con selenium para que puedas acceder y resolver un captcha, luego de esto, deberas escribir el input,
        "y" si lo hAS resuelto o "n" si ha ocurrido algun error. Puedes usar "break" para finalizar el proceso.

    Parámetros:
    - url: str de una url

    Salida:
    - captcha: str, "y" o "n", "break para finalizar el proceso"
    """
    driver = webdriver.Chrome(options=option)
    driver.get(url)
    captcha=input('Captcha resuelto?y/n')
    driver.quit()

    return captcha

In [40]:
start=time.time()
resultados=[]
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

for ind,url in enumerate(df['urls'].values[:]):
    response = requests.get(url, headers=headers)
    # Verifica si la solicitud fue exitosa
    if response.status_code == 200:
        # Extraer los resultados para cada alojamiento y añadirlos a la lista
        try:
            titulo, lat, long, loc, localidad, precios, telefono, horario, tipo_cocina, valora,votos,caracteristicas, descripcion = from_responseContent_to_data(response.content)
            #Verifica que tenemos latitud y longitud
            if titulo !='KO':
                resultados.append({
                    'Nombre': titulo, 'Localización': localidad, 'Latitud': lat, 'Longitud': long,
                    'Dirección': loc, "Teléfono": telefono, "Precio": precios, 'Horario': horario,
                    "Tipos de cocina": tipo_cocina, "Valoración": valora, "Número de votos": votos, 
                    "Características": caracteristicas, "Descripción": descripcion
            })
                print(f'URL OK ({ind}): {url}')
            else:
                print(f"\033[33mError no se han obtenido coordenadas en la URL ({ind}): {url}\033[0m")
        # si hay error seguramente sea por cpcha--->resolvemos manualmente
        except:
            print(f'\033[91mError en la URL ({ind}): {url}\033[0m')
            captcha=manual_captcha_resolve(url)
            #una vez resuelto obtenemos los datos
            if captcha=='y':
                response = requests.get(url, headers=headers)
                titulo, lat, long, loc, localidad, precios, telefono, horario, tipo_cocina, valora,votos,caracteristicas, descripcion = from_responseContent_to_data(response.content)
                resultados.append({
                    'Nombre': titulo, 'Localización': localidad, 'Latitud': lat, 'Longitud': long,
                    'Dirección': loc, "Teléfono": telefono, "Precio": precios, 'Horario': horario,
                    "Tipos de cocina": tipo_cocina, "Valoración": valora, "Número de votos": votos, 
                    "Características": caracteristicas, "Descripción": descripcion
            })
                print(f"\033[32mCaptcha resuelto, URL OK ({ind})\033[0m")
            #si queremos finalizar el proceso
            elif captcha=='break':
                break
            #si ha ocurrido algun error al resolver pasamos al siguiente
            else:
                continue
    #si la solicitud no fue existosa
    else:
        print(f"\033[91mError al acceder a la URL. Código de estado: {response.status_code}\033[0m")
df_resultados = pd.DataFrame(resultados)
print(f'finalizado en {round((time.time()-start)/60,2)}min')

Error en la URL (0): https://es.restaurantguru.com/Sidreria-Narcea-Cangas-del-Narcea
Captcha resuelto, URL OK (0)
URL OK (1): https://es.restaurantguru.com/Restaurante-Casa-Del-Rio-Cangas-del-Narcea
URL OK (2): https://es.restaurantguru.com/Roble-Pola-de-Lena
URL OK (3): https://es.restaurantguru.com/Monte-San-Feliz-2
URL OK (4): https://es.restaurantguru.com/El-Molin-de-la-Pedrera-Cangas-de-Onis
URL OK (5): https://es.restaurantguru.com/Casa-Pedro-Parres-San-Juan-de-Parres
URL OK (6): https://es.restaurantguru.com/Cafe-Pandora-Aviles
URL OK (7): https://es.restaurantguru.com/Restaurante-Gunea-Illas
URL OK (8): https://es.restaurantguru.com/Autobar-Restaurante-Grado-Asturias
URL OK (9): https://es.restaurantguru.com/Casa-Pepe-El-Bueno-Grado-Asturias
Error en la URL (10): https://es.restaurantguru.com/Abrelatas-Restaurante-Pola-de-Siero
Captcha resuelto, URL OK (10)
URL OK (11): https://es.restaurantguru.com/El-Polesu-Pola-de-Siero
URL OK (12): https://es.restaurantguru.com/Casa-Vicente

In [43]:
df_resultados.tail()

,Nombre,Localización,Latitud,Longitud,Dirección,Teléfono,Precio,Horario,Tipos de cocina,Valoración,Número de votos,Características,Descripción
40,Restaurante El Cafetin,Lastres,43.51282860,-5.26870180,"Calle Matematico Pedrayes, Lastres, Principado...",+34679817804,€10 - €24,"[LunesLun, 11:00-16:00, MartesMar, Cerrado, Mi...",[Española],92,975,"[Asientos al aire libre, Para llevar, Tarjetas...",Es estupendo poder probar la cocina española. ...
41,Sidrería Casa Niembro,Asiego,43.32592810,-4.86341920,"Asiegu sn, Asiego, Principado de Asturias, España",+34985845001,€19 - €24,"[LunesLun, Cerrado, MartesMar, Cerrado, Miérco...","[Española, Opciones vegetarianas]",92,2456,"[Asientos al aire libre, Tarjetas de crédito a...",La comida española merece aquí la pena. En est...
42,Sidrería Ribeles,Las Arenas,43.30268470,-4.81610190,"AS-114, 6624, Las Arenas, Principado de Asturi...",+34635324979,€10 - €24,"[LunesLun, Cerrado, MartesMar, 12:00-17:00, Mi...","[Mediterránea, Española, Opciones vegetarianas]",90,1600,"[Asientos al aire libre, Para llevar, Tarjetas...",Los turistas disfrutan de los platos de las co...
43,A Reigada Hotel Restaurante,Grandas de Salime,43.21686580,-6.87507530,"C. Pedro de Pedre, 9, Grandas de Salime, Princ...",+34985627017,hasta €10,"[LunesLun, 07:00-16:0019:00-23:00, MartesMar, ...",[Española],80,1219,"[Tarjetas de crédito aceptadas, No hay entrega...",Una vez que hayas visto Ethnografic Museum de ...
44,Restaurante Bar La Parrilla Cereijeira,Cereijeira,43.21148970,-6.90427820,"33730 Asturias, Aldea Cereijeira, 13A, Cereije...",+34985627056,€10 - €24,"[LunesLun, 07:00-16:00, MartesMar, 07:00-16:00...","[Española, Asado, Grill]",78,393,"[Asientos al aire libre, Tarjetas de crédito a...",Los clientes disfrutan de los platos de la coc...


In [ ]:
# df_resultados.to_csv('../../Data/Data_used/restaurant_test.csv', index=False)
